In [13]:
# with hugging face wrapper for sanity check
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from ast import literal_eval
import re

In [ ]:
#training hyperparameters
MAX_TOKENS = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05

model_checkpoint = "distilbert/distilbert-base-uncased"

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [15]:
review_df = pd.read_csv('../data/split/train.csv', usecols=['Overall Compliance', 'reviews'])
test_df = pd.read_csv('../data/split/test.csv', usecols=['Overall Compliance', 'reviews'])
review_df['reviews'] = review_df['reviews'].apply(literal_eval)
test_df['reviews'] = test_df['reviews'].apply(literal_eval)

train_ds = Dataset.from_pandas(review_df)
test_df = Dataset.from_pandas(test_df)

review_dataset = DatasetDict()

review_dataset['train'] = train_ds
review_dataset['test'] = test_df

In [11]:
def clean_reviews(reviews):
    cleaned = []
    for review in reviews:
        review = review.replace('\n', ' ')
        cleaned.append(re.sub(r"[^a-zA-Z0-9]", ' ', review).strip()) #may need to find a better way to do so

    return cleaned

def extract_text_features(data):
    output = {}
    reviews = clean_reviews(data['reviews'])
    output['text'] = " ".join(reviews)
    output['label'] = 0 if data['Overall Compliance'] == 'Yes' else 0

    return output

In [16]:
review_dataset = review_dataset.map(extract_text_features)

Map: 100%|██████████| 195/195 [00:00<00:00, 6425.45 examples/s]


In [29]:
review_dataset['train'][0].keys()

dict_keys(['Overall Compliance', 'reviews', 'text', 'label'])

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize(examples):
    return tokenizer(examples["text"], max_length=MAX_TOKENS, truncation=True)

In [24]:
token_dataset = review_dataset.map(tokenize, remove_columns=['Overall Compliance', 'reviews'])

Map: 100%|██████████| 195/195 [00:00<00:00, 541.34 examples/s]


In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [26]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [27]:
id2label = {0: "Pass", 1: "Fail"}
label2id = {"Pass": 0, "Fail": 1}

In [28]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_dataset["train"],
    eval_dataset=token_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
